In [1]:
import emoji

In [2]:
len(emoji.EMOJI_UNICODE)

2811

In [3]:
emoji_dictionary = {
    "0":"\u2764\uFE0F",
    "1":":baseball:",
    "2":":grinning_face_with_big_eyes:",
    "3":":disappointed_face:",
    "4":":fork_and_knife:",
}

In [4]:
for e in emoji_dictionary.values():
    print(emoji.emojize(e))

❤️
⚾
😃
😞
🍴


# Processing Dataset

In [5]:
import pandas as pd
import numpy as np

In [6]:
train = pd.read_csv('train_emoji.csv',header = None)
test = pd.read_csv('test_emoji.csv', header = None)

In [7]:
test.head()

,0,1
0,I want to eat\t,4
1,he did not answer\t,3
2,he got a raise\t,2
3,she got me a present\t,0
4,ha ha ha it was so funny\t,2


In [8]:
train.head()

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN


In [9]:
data = train.values
print(data.shape)

(132, 4)


In [10]:
X_train = train[0]
Y_train = train[1]

X_test = test[0]
Y_test = test[1]

In [11]:
for i in range(5):
    print(X_train[i],emoji.emojize(emoji_dictionary[str(Y_train[i])]))

never talk to me again 😞
I am proud of your achievements 😃
It is the worst day in my life 😞
Miss you so much ❤️
food is life 🍴


# converting sentences using Embeddings using Glove

In [12]:
f = open('Datasets/glove6b50dtxt/glove.6B.50d.txt',encoding='utf-8')

In [13]:
embeddings_index = {}
cnt=0
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:],dtype='float')
    embeddings_index[word] = coefs

In [14]:
f.close()

In [15]:
emb_dim = embeddings_index['eat'].shape[0]

In [16]:
print(emb_dim)

50


# converting sentence into vectors

In [17]:
def embedding_output_train(X):
    maxLen = 10
    embedding_out = np.zeros((X.shape[0],maxLen,emb_dim))
    
    for ix in range(X.shape[0]):
        X[ix] = X[ix].split()
        for ij in range(len(X[ix])):
            try:
                embedding_out[ix][ij] = embeddings_index[X[ix][ij].lower()]
            except:
                embedding_out[ix][ij] = np.zeros((50,))
    return embedding_out
            

In [18]:
embeddings_matrix_train = embedding_output_train(X_train)


/home/insanenerd/myvenv/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [19]:
embeddings_matrix_test = embedding_output_train(X_test)

/home/insanenerd/myvenv/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [20]:
print(embeddings_matrix_test.shape,embeddings_matrix_train.shape)

(56, 10, 50) (132, 10, 50)


In [21]:
from keras.utils import to_categorical

Using TensorFlow backend.


In [22]:
Y_train = to_categorical(Y_train,num_classes=5)
Y_test = to_categorical(Y_test,num_classes=5)
print(Y_train.shape)
print(Y_train[0])

(132, 5)
[0. 0. 0. 1. 0.]


# Define RNN/LSTM MODEL

In [23]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential

In [50]:
model = Sequential()
model.add(LSTM(64,input_shape=(10,emb_dim)))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 64)                29440     
_________________________________________________________________
dropout_4 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 325       
_________________________________________________________________
activation_4 (Activation)    (None, 5)                 0         
Total params: 29,765
Trainable params: 29,765
Non-trainable params: 0
_________________________________________________________________


In [51]:
from keras.callbacks import EarlyStopping,ModelCheckpoint
checkpoint = ModelCheckpoint("best_model.h5",monitor='val_loss',verbose=True,save_best_only=True)
earlystop = EarlyStopping(monitor='val_acc',patience=10)
hist = model.fit(embeddings_matrix_train,Y_train,epochs=100,batch_size=64,shuffle=True,validation_split=0.2,callbacks=[earlystop,checkpoint])

Train on 105 samples, validate on 27 samples
Epoch 1/100
 64/105 [=================>............] - ETA: 0s - loss: 1.6158 - accuracy: 0.2656
Epoch 00001: val_loss improved from inf to 1.60425, saving model to best_model.h5
105/105 [==============================] - 2s 16ms/sample - loss: 1.6101 - accuracy: 0.2476 - val_loss: 1.6043 - val_accuracy: 0.1852
Epoch 2/100
 64/105 [=================>............] - ETA: 0s - loss: 1.5614 - accuracy: 0.3125
Epoch 00002: val_loss improved from 1.60425 to 1.59984, saving model to best_model.h5
105/105 [==============================] - 0s 438us/sample - loss: 1.5645 - accuracy: 0.3333 - val_loss: 1.5998 - val_accuracy: 0.1852
Epoch 3/100
 64/105 [=================>............] - ETA: 0s - loss: 1.5533 - accuracy: 0.2969
Epoch 00003: val_loss did not improve from 1.59984
105/105 [==============================] - 0s 288us/sample - loss: 1.5547 - accuracy: 0.3048 - val_loss: 1.6004 - val_accuracy: 0.1481
Epoch 4/100
 64/105 [=================>..

/home/insanenerd/myvenv/lib/python3.6/site-packages/keras/callbacks/callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning



Epoch 00006: val_loss did not improve from 1.59984
105/105 [==============================] - 0s 384us/sample - loss: 1.4990 - accuracy: 0.4286 - val_loss: 1.6127 - val_accuracy: 0.1481
Epoch 7/100
 64/105 [=================>............] - ETA: 0s - loss: 1.4679 - accuracy: 0.3750
Epoch 00007: val_loss did not improve from 1.59984
105/105 [==============================] - 0s 481us/sample - loss: 1.4746 - accuracy: 0.3810 - val_loss: 1.6194 - val_accuracy: 0.1481
Epoch 8/100
 64/105 [=================>............] - ETA: 0s - loss: 1.4464 - accuracy: 0.4531
Epoch 00008: val_loss did not improve from 1.59984
105/105 [==============================] - 0s 349us/sample - loss: 1.4663 - accuracy: 0.4286 - val_loss: 1.6248 - val_accuracy: 0.1852
Epoch 9/100
 64/105 [=================>............] - ETA: 0s - loss: 1.4337 - accuracy: 0.4062
Epoch 00009: val_loss did not improve from 1.59984
105/105 [==============================] - 0s 242us/sample - loss: 1.4344 - accuracy: 0.4095 - val_

Epoch 33/100
 64/105 [=================>............] - ETA: 0s - loss: 0.5579 - accuracy: 0.7812
Epoch 00033: val_loss improved from 0.95087 to 0.93815, saving model to best_model.h5
105/105 [==============================] - 0s 439us/sample - loss: 0.5014 - accuracy: 0.8381 - val_loss: 0.9381 - val_accuracy: 0.5926
Epoch 34/100
 64/105 [=================>............] - ETA: 0s - loss: 0.5031 - accuracy: 0.8438
Epoch 00034: val_loss did not improve from 0.93815
105/105 [==============================] - 0s 222us/sample - loss: 0.4350 - accuracy: 0.8571 - val_loss: 0.9600 - val_accuracy: 0.5926
Epoch 35/100
 64/105 [=================>............] - ETA: 0s - loss: 0.4448 - accuracy: 0.8438
Epoch 00035: val_loss did not improve from 0.93815
105/105 [==============================] - 0s 236us/sample - loss: 0.4261 - accuracy: 0.8762 - val_loss: 0.9986 - val_accuracy: 0.6296
Epoch 36/100
 64/105 [=================>............] - ETA: 0s - loss: 0.3651 - accuracy: 0.8594
Epoch 00036: va

Epoch 62/100
 64/105 [=================>............] - ETA: 0s - loss: 0.0531 - accuracy: 1.0000
Epoch 00062: val_loss did not improve from 0.93815
105/105 [==============================] - 0s 261us/sample - loss: 0.0678 - accuracy: 0.9905 - val_loss: 1.2692 - val_accuracy: 0.6296
Epoch 63/100
 64/105 [=================>............] - ETA: 0s - loss: 0.0876 - accuracy: 1.0000
Epoch 00063: val_loss did not improve from 0.93815
105/105 [==============================] - 0s 279us/sample - loss: 0.0977 - accuracy: 0.9810 - val_loss: 1.1987 - val_accuracy: 0.6667
Epoch 64/100
 64/105 [=================>............] - ETA: 0s - loss: 0.0271 - accuracy: 1.0000
Epoch 00064: val_loss did not improve from 0.93815
105/105 [==============================] - 0s 273us/sample - loss: 0.0394 - accuracy: 1.0000 - val_loss: 1.2010 - val_accuracy: 0.5926
Epoch 65/100
 64/105 [=================>............] - ETA: 0s - loss: 0.1160 - accuracy: 0.9531
Epoch 00065: val_loss did not improve from 0.93815

Epoch 91/100
 64/105 [=================>............] - ETA: 0s - loss: 0.0174 - accuracy: 1.0000
Epoch 00091: val_loss did not improve from 0.93815
105/105 [==============================] - 0s 259us/sample - loss: 0.0158 - accuracy: 1.0000 - val_loss: 1.3586 - val_accuracy: 0.6296
Epoch 92/100
 64/105 [=================>............] - ETA: 0s - loss: 0.0162 - accuracy: 1.0000
Epoch 00092: val_loss did not improve from 0.93815
105/105 [==============================] - 0s 215us/sample - loss: 0.0186 - accuracy: 1.0000 - val_loss: 1.3647 - val_accuracy: 0.6296
Epoch 93/100
 64/105 [=================>............] - ETA: 0s - loss: 0.0215 - accuracy: 1.0000
Epoch 00093: val_loss did not improve from 0.93815
105/105 [==============================] - 0s 248us/sample - loss: 0.0178 - accuracy: 1.0000 - val_loss: 1.3393 - val_accuracy: 0.6296
Epoch 94/100
 64/105 [=================>............] - ETA: 0s - loss: 0.0140 - accuracy: 1.0000
Epoch 00094: val_loss did not improve from 0.93815

In [52]:
model.load_weights("best_model.h5")

In [53]:
pred = model.predict_classes(embeddings_matrix_test)

In [54]:
print(pred)

[4 3 3 0 2 2 3 2 4 2 1 2 0 3 1 3 2 2 3 2 0 3 4 0 3 3 3 0 4 2 0 1 0 2 3 3 2
 3 1 2 1 0 0 1 2 3 2 2 3 1 3 0 3 2 2 3]


In [55]:
model.evaluate(embeddings_matrix_test,Y_test)

56/56 [==============================] - 0s 80us/sample - loss: 1.0245 - accuracy: 0.5893


[1.024472645350865, 0.58928573]

# Stacked LSTM 

In [72]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential

In [73]:
model = Sequential()
model.add(LSTM(64,input_shape=(10,emb_dim),return_sequences=True))
model.add(Dropout(0.6))
model.add(LSTM(64,input_shape=(10,emb_dim),return_sequences=False))
model.add(Dropout(0.6))
model.add(Dense(5))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_13 (LSTM)               (None, 10, 64)            29440     
_________________________________________________________________
dropout_13 (Dropout)         (None, 10, 64)            0         
_________________________________________________________________
lstm_14 (LSTM)               (None, 64)                33024     
_________________________________________________________________
dropout_14 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 5)                 325       
_________________________________________________________________
activation_9 (Activation)    (None, 5)                 0         
Total params: 62,789
Trainable params: 62,789
Non-trainable params: 0
__________________________________________________

In [74]:
from keras.callbacks import EarlyStopping,ModelCheckpoint
checkpoint = ModelCheckpoint("best_model1.h5",monitor='val_loss',verbose=True,save_best_only=True)
earlystop = EarlyStopping(monitor='val_acc',patience=10)
hist = model.fit(embeddings_matrix_train,Y_train,epochs=150,batch_size=64,shuffle=True,validation_split=0.2)

Train on 105 samples, validate on 27 samples
Epoch 1/150
105/105 [==============================] - 2s 19ms/sample - loss: 1.6225 - accuracy: 0.1810 - val_loss: 1.5934 - val_accuracy: 0.2222
Epoch 2/150
105/105 [==============================] - 0s 312us/sample - loss: 1.5933 - accuracy: 0.2286 - val_loss: 1.5869 - val_accuracy: 0.2222
Epoch 3/150
105/105 [==============================] - 0s 364us/sample - loss: 1.5583 - accuracy: 0.3143 - val_loss: 1.5815 - val_accuracy: 0.2222
Epoch 4/150
105/105 [==============================] - 0s 346us/sample - loss: 1.5233 - accuracy: 0.3714 - val_loss: 1.5795 - val_accuracy: 0.2222
Epoch 5/150
105/105 [==============================] - 0s 307us/sample - loss: 1.4977 - accuracy: 0.2762 - val_loss: 1.5807 - val_accuracy: 0.2222
Epoch 6/150
105/105 [==============================] - 0s 268us/sample - loss: 1.4965 - accuracy: 0.3905 - val_loss: 1.5845 - val_accuracy: 0.2222
Epoch 7/150
105/105 [==============================] - 0s 316us/sample - l

Epoch 53/150
105/105 [==============================] - 0s 321us/sample - loss: 0.0900 - accuracy: 0.9810 - val_loss: 0.9889 - val_accuracy: 0.6667
Epoch 54/150
105/105 [==============================] - 0s 278us/sample - loss: 0.1166 - accuracy: 0.9714 - val_loss: 1.1419 - val_accuracy: 0.6296
Epoch 55/150
105/105 [==============================] - 0s 234us/sample - loss: 0.1136 - accuracy: 0.9810 - val_loss: 1.3727 - val_accuracy: 0.5556
Epoch 56/150
105/105 [==============================] - 0s 245us/sample - loss: 0.1027 - accuracy: 0.9714 - val_loss: 1.1428 - val_accuracy: 0.6667
Epoch 57/150
105/105 [==============================] - 0s 234us/sample - loss: 0.1150 - accuracy: 0.9714 - val_loss: 1.2704 - val_accuracy: 0.6667
Epoch 58/150
105/105 [==============================] - 0s 234us/sample - loss: 0.1984 - accuracy: 0.9619 - val_loss: 1.3285 - val_accuracy: 0.5926
Epoch 59/150
105/105 [==============================] - 0s 250us/sample - loss: 0.1023 - accuracy: 0.9810 - val_

In [66]:
#model.load_weights("best_model1.h5")

In [75]:
pred = model.predict_classes(embeddings_matrix_test)

In [80]:
print(len(pred))

56


In [77]:
model.evaluate(embeddings_matrix_test,Y_test)

56/56 [==============================] - 0s 119us/sample - loss: 2.5910 - accuracy: 0.6071


[2.590971793447222, 0.60714287]

In [79]:
for i in range(len(pred)):
    print(' '.join(X_test[i]))
    print(emoji.emojize(emoji_dictionary[str(np.argmax(Y_test[i]))]))
    print(emoji.emojize(emoji_dictionary[str(pred[i])]))

I want to eat
🍴
🍴
he did not answer
😞
😞
he got a raise
😃
😞
she got me a present
❤️
😞
ha ha ha it was so funny
😃
😃
he is a good friend
❤️
😃
I am upset
❤️
😞
We had such a lovely dinner tonight
❤️
😃
where is the food
🍴
🍴
Stop making this joke ha ha ha
😃
😃
where is the ball
⚾
⚾
work is hard
😞
😃
This girl is messing with me
😞
😞
are you serious ha ha
😃
😞
Let us go play baseball
⚾
⚾
This stupid grader is not working
😞
😞
work is horrible
😞
😃
Congratulation for having a baby
😃
😃
stop messing around
😞
😞
any suggestions for dinner
🍴
😃
I love taking breaks
❤️
😞
you brighten my day
😃
❤️
I boiled rice
🍴
🍴
she is a bully
😞
😃
Why are you feeling bad
😞
😞
I am upset
😞
😞
I worked during my birthday
😞
😃
My grandmother is the love of my life
❤️
❤️
enjoy your break
😃
⚾
valentine day is near
❤️
😃
I miss you so much
❤️
❤️
throw the ball
⚾
⚾
My life is so boring
😞
😞
she said yes
😃
😃
will you be my valentine
❤️
❤️
he can pitch really well
⚾
⚾
dance with me
😃
😃
I am starving
🍴
🍴
See you at the restaurant
🍴
😃
I l